In [0]:
google_colab = True
if google_colab:
  %tensorflow_version 2.x

In [0]:
if google_colab:
  from google.colab import drive
  drive.mount('/content/drive')

In [0]:
if google_colab:
  !unzip ann-and-dl-image-segmentation.zip

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Image Segmentation

In [11]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            validation_split=0.2,
                                            rescale=1./255)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             cval=0,
                                             validation_split=0.2,
                                             rescale=1./255)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255)
    train_mask_data_gen = ImageDataGenerator()

# Create validation and test ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator()
test_img_data_gen = ImageDataGenerator(rescale=1./255)
test_mask_data_gen = ImageDataGenerator()

In [12]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')

# Batch size
bs = 64

# img shape
img_h = 256
img_w = 256

num_classes=2

training_dir = os.path.join(dataset_dir, 'training')
# Training
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None,
                                                       shuffle=True,
                                                       subset='training',
                                                       interpolation='bilinear',
                                                       color_mode='rgb',
                                                       seed=SEED)  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None,
                                                         shuffle=True,
                                                         subset='training',
                                                         interpolation='bilinear',
                                                         color_mode='grayscale',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None,
                                                       shuffle=False,
                                                       subset='validation',
                                                       interpolation='bilinear',
                                                       color_mode='rgb',
                                                       seed=SEED)
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None,
                                                         shuffle=False,
                                                         subset='validation',
                                                         interpolation='bilinear',
                                                         color_mode='grayscale',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)

# Test
test_dir = os.path.join(dataset_dir, 'test')
test_img_gen = test_img_data_gen.flow_from_directory(os.path.join(test_dir, 'images'),
                                                     target_size=(img_h, img_w),
                                                     batch_size=bs, 
                                                     class_mode=None,
                                                     shuffle=False,
                                                     interpolation='bilinear',
                                                     seed=SEED)

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\riccardo\\Documents\\Scuola\\5anno\\AN2DL\\A2NDL_Challange\\Challenge2\\Segmentation_Dataset\\training\\images'

In [0]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.int32)
    return x_, tf.where(y_ > 0, y_ - 1, y_ + 1)

train_dataset = train_dataset.map(prepare_target)

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
test_dataset = test_dataset.map(prepare_target)

# Repeat
test_dataset = valid_dataset.repeat()

## Convolutional Neural Network (CNN)
### Encoder-Decoder

In [0]:
# Create Model
# ------------

def create_model(depth, start_f, num_classes, dynamic_input_shape):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    for i in range(depth):
        
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, 3]
            else:
                input_shape = [img_h, img_w, 3]
        else:
            input_shape=[None]
        
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f // 2,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))

        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='softmax'))
    
    return model

In [0]:
model = create_model(depth=4, 
                     start_f=4, 
                     num_classes=2, 
                     dynamic_input_shape=False)

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 256, 256, 4)       112       
_________________________________________________________________
re_lu_16 (ReLU)              (None, 256, 256, 4)       0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 128, 128, 4)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 128, 128, 8)       296       
_________________________________________________________________
re_lu_17 (ReLU)              (None, 128, 128, 8)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 64, 64, 16)       

[<tf.Variable 'conv2d_18/kernel:0' shape=(3, 3, 3, 4) dtype=float32, numpy=
 array([[[[ 2.34783888e-02, -8.35591853e-02,  5.03832102e-02,
           -1.58112541e-01],
          [-4.29977775e-02,  2.50706702e-01,  4.45672870e-03,
           -8.71395469e-02],
          [ 1.72109574e-01,  5.26496172e-02,  1.65915161e-01,
           -1.03339851e-02]],
 
         [[ 5.39380312e-02, -2.94595838e-01, -3.97144556e-02,
            2.95175880e-01],
          [ 2.96923131e-01, -2.18196660e-01,  3.05256993e-01,
            1.73660725e-01],
          [-1.72739327e-01, -6.79370761e-03,  8.21659267e-02,
            3.93394232e-02]],
 
         [[-5.94584048e-02, -6.25239462e-02,  1.12761170e-01,
            1.69196188e-01],
          [-3.36254537e-02, -2.73518294e-01, -1.56801611e-01,
           -1.20162189e-01],
          [-1.12727687e-01, -1.89474687e-01,  2.00690955e-01,
            5.69806695e-02]]],
 
 
        [[[-7.75248706e-02,  2.11911589e-01, -1.60213262e-01,
            1.98817581e-01],
  

## Prepare the model for training

In [0]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------
def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

metrics = [my_IoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training with callbacks

In [0]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=30,
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

Train for 96 steps, validate for 24 steps
Epoch 1/30
96/96 [==============================] - 124s 1s/step - loss: 0.5516 - my_IoU: 0.7429 - val_loss: 0.4877 - val_my_IoU: 0.7432
Epoch 2/30
96/96 [==============================] - 123s 1s/step - loss: 0.4740 - my_IoU: 0.7428 - val_loss: 0.4525 - val_my_IoU: 0.7427
Epoch 3/30
96/96 [==============================] - 123s 1s/step - loss: 0.4534 - my_IoU: 0.7421 - val_loss: 0.4341 - val_my_IoU: 0.7432
Epoch 4/30
96/96 [==============================] - 124s 1s/step - loss: 0.4371 - my_IoU: 0.7425 - val_loss: 0.4462 - val_my_IoU: 0.7405
Epoch 5/30
96/96 [==============================] - 127s 1s/step - loss: 0.4249 - my_IoU: 0.7423 - val_loss: 0.4383 - val_my_IoU: 0.7432
Epoch 6/30
96/96 [==============================] - 124s 1s/step - loss: 0.4180 - my_IoU: 0.7434 - val_loss: 0.4233 - val_my_IoU: 0.7430
Epoch 7/30
96/96 [==============================] - 122s 1s/step - loss: 0.4177 - my_IoU: 0.7428 - val_loss: 0.4133 - val_my_IoU: 0.7429

## Test model

In [0]:
restore_model = False
if restore_model:
    model = create_model(depth=4, 
                         start_f=4, 
                         num_classes=3, 
                         dynamic_input_shape=True)

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                  metrics=['accuracy'])

    model.load_weights(os.path.join(
        cwd, 'segmentation_experiments', 'EXP_FOLDER', 'ckpts', 'cp_60.ckpt'))

eval_out = model.evaluate(x=test_dataset,
                          steps=len(test_img_gen),
                          verbose=0)

eval_out

## Compute prediction

In [6]:
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [7]:
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

In [14]:
import time
import PIL

from PIL import Image

# Cycle over test images

test_img_dir = os.path.join(test_dir, 'images', 'img')

img_filenames = next(os.walk(test_img_dir))[2]

results={}

for img_filename in img_filenames:
    
    img_filename = img_filename[:-4]
    img = Image.open(os.path.join(test_img_dir, img_filename))
    img = img.resize((img_h, img_w))
    
    img_arr = np.expand_dims(np.array(img), 0)
    
    out_softmax = model.predict(x=img_arr / 255.)
    
    # Get predicted class as the index corresponding to the maximum value in the vector probability
    predicted_class = tf.argmax(out_softmax, -1)
    predicted_class = predicted_class[0]

    target = np.array(mask)
    target -= 1
    prediction_img = np.zeros([256, 256, 1])
    prediction_img[np.where(target == 0)] = [0]
    prediction_img[np.where(target == 1)] = [1]
    results[img_filename] = rle_encode(prediction_img)

create_csv(results)

ModuleNotFoundError: No module named 'PIL'